In [1]:
import pandas as pd
import numpy as np
import glob
import datetime
#import geopandas as gp

In [2]:
#pull all files and append to a list
path = r'C:\Users\Erin\Documents\nss-data-analytics\projects\mapping-mexico\data\icews' # use your path
all_files = glob.glob(path + "/*.tab")

li = []

for filename in all_files:
    df = pd.read_csv(filename, sep='\t', index_col=None, header=0, parse_dates=['Event Date'],dtype={
       'Event ID':'int','Intensity':'float','Story ID':'int','Sentence Number':'int'})
    li.append(df)

C:\Users\Erin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#concatenate lists to one dataframe
icews_clean = pd.concat(li, axis=0, ignore_index=True)

In [4]:
icews_clean.head()

,Event ID,Event Date,Source Name,Source Sectors,Source Country,Event Text,CAMEO Code,Intensity,Target Name,Target Sectors,Target Country,Story ID,Sentence Number,Publisher,City,District,Province,Country,Latitude,Longitude
0,926685,1995-01-01,Extremist (Russia),"Radicals / Extremists / Fundamentalists,Dissident",Russian Federation,Praise or endorse,51,3.4,Boris Yeltsin,"Elite,Executive,Executive Office,Government",Russian Federation,28235806,5,The Toronto Star,Moscow,NaN,Moskva,Russian Federation,55.7522,37.6156
1,926687,1995-01-01,Government (Bosnia and Herzegovina),Government,Bosnia and Herzegovina,Express intent to cooperate,30,4.0,Citizen (Serbia),"General Population / Civilian / Social,Social",Serbia,28235807,1,The Toronto Star,NaN,NaN,Bosnia,Bosnia and Herzegovina,44.0000,18.0000
2,926686,1995-01-01,Citizen (Serbia),"General Population / Civilian / Social,Social",Serbia,Express intent to cooperate,30,4.0,Government (Bosnia and Herzegovina),Government,Bosnia and Herzegovina,28235807,1,The Toronto Star,NaN,NaN,Bosnia,Bosnia and Herzegovina,44.0000,18.0000
3,926688,1995-01-01,Canada,NaN,Canada,Praise or endorse,51,3.4,City Mayor (Canada),"Government,Local,Municipal",Canada,28235809,3,The Toronto Star,NaN,NaN,Ontario,Canada,49.2501,-84.4998
4,926689,1995-01-01,Lawyer/Attorney (Canada),"Legal,Social",Canada,"Arrest, detain, or charge with legal action",173,-5.0,Police (Canada),"Government,Police",Canada,28235964,1,The Toronto Star,Montreal,Montreal,Quebec,Canada,45.5088,-73.5878


In [5]:
#rename the columns
icews_clean = icews_clean.rename({'Event ID':'event_id', 'Event Date':'event_date', 'Source Name':'source_name',
    'Source Sectors':'source_sectors','Source Country':'source_country', 'Event Text':'event_text',
    'CAMEO Code':'cameo_code', 'Intensity':'intensity','Target Name':'target_name','Target Sectors':'target_sectors',
    'Target Country':'target_country', 'Story ID':'story_id', 'Sentence Number':'sentence_number',
    'Publisher':'publisher', 'City':'city', 'District':'district', 'Province':'provice',
    'Country':'country', 'Latitude':'lat', 'Longitude':'lon'}, axis=1)

In [6]:
#filter for country Mexico
mex_clean = icews_clean.loc[icews_clean["country"].isin(['Mexico'])]

In [7]:
mex_clean.shape

(195581, 20)

In [8]:
#checking for character issues w/ accents
explore = mex_clean['source_name'].unique()
print(explore.tolist())

['Protester (Mexico)', 'Police (Mexico)', 'Andrés Manuel López Obrador', 'Main Opposition (Mexico)', 'National Action Party (Mexico)', 'Party of the Democratic Revolution', 'Citizen (Mexico)', 'O Estado de S. Paulo', 'Armed Band (Brazil)', 'Porfirio Muñoz Ledo', 'Mexico', 'European Union', 'Congress (Mexico)', 'Representatives (Mexico)', 'Rebel Commander (Mexico)', 'Bishop (Mexico)', 'Political Parties (Mexico)', 'Warren Christopher', 'People Associated with the Opposition (Mexico)', 'Newspaper (Mexico)', 'Rioter (Mexico)', 'Party Member (Mexico)', 'Activist (Institutional Revolutionary Party)', 'Institutional Revolutionary Party', 'Ernesto Zedillo', 'Newt Gingrich', 'Bill Clinton', 'Michel Camdessus', 'George W. Bush', 'Democratic Revolutionary Party', 'Zapatista Army of National Liberation', 'Ecuador', 'Peru', 'Attacker (Mexico)', 'Mike McCurry', 'Cuauhtémoc Cárdenas', 'Military (Mexico)', 'Government Official (Mexico)', 'Eduardo Robledo Rincón', 'Subcomandante Marcos', 'Business (Me

In [9]:
#character cleaning
typos = {
    'Ã\x81':'Á',
    'Ã¡':'á',
    'Ã©':'é',
    'Ã\xad':'í',
    'Ã³':'ó',
    'Ãº':'ú',
    'Ã±':'ñ'}
mex_clean = mex_clean.replace(typos)

In [10]:
#CAMEO codes to show which events are present in the dataframe
explore = mex_clean['cameo_code'].unique()
explore

array([1411, 172, 124, 14, 130, 36, 112, 192, 10, 193, 57, 160, 40, 51,
       100, 144, 111, 141, 190, 1721, 145, 20, 1041, 311, 43, 42, 27, 871,
       150, 32, 1013, 173, 54, 113, 35, 120, 874, 16, 90, 180, 12, 841,
       46, 181, 62, 30, 22, 41, 127, 154, 19, 71, 831, 45, 174, 81, 13,
       202, 1122, 353, 175, 128, 104, 106, 133, 214, 50, 91, 61, 161,
       1043, 17, 115, 182, 70, 351, 251, 143, 331, 60, 53, 142, 1121, 138,
       63, 125, 80, 241, 39, 72, 74, 861, 75, 233, 33, 94, 1711, 1412,
       164, 73, 152, 1453, 52, 165, 1313, 191, 170, 1242, 332, 1822, 1246,
       163, 1821, 213, 313, 18, 132, 15, 37, 186, 21, 23, 121, 833, 131,
       1243, 194, 153, 102, 108, 114, 137, 28, 38, 26, 11, 256, 44, 195,
       64, 356, 24, 1823, 341, 1312, 834, 1014, 151, 842, 101, 1056, 171,
       1621, 1125, 242, 92, 1712, 107, 1213, 1661, 1042, 134, 231, 126,
       183, 333, 1241, 862, 55, 832, 1044, 1722, 31, 105, 139, 873, 253,
       1724, 83, 25, 342, 1432, 243, 1623, 1031, 34, 

In [33]:
#fix 2017 cameo codes
cameo = {
    '180':180,
    '181':181,
    '182':182,
    '183':183,
    '184':184,
    '185':185,
    '186':186,
    '190':190,
    '191':191,
    '192':192,
    '193':193,
    '194':194,
    '195':195,
    '202':202,
    '1822':1822,
    '128':128,
    '1823':1823}
mex_clean = mex_clean.replace(cameo)

In [34]:
mex_clean.head(10)

,event_id,event_date,source_name,source_sectors,source_country,event_text,cameo_code,intensity,target_name,target_sectors,target_country,story_id,sentence_number,publisher,city,district,provice,country,lat,lon
12,926701,1995-01-01,Protester (Mexico),"Protestors / Popular Opposition / Mobs,Dissident",Mexico,Demonstrate for leadership change,1411,-6.5,Governor (Mexico),"Government,Provincial,Local",Mexico,28238094,1,The Associated Press Political Service,Tabasco,Jose Maria Morelos,Estado de Quintana Roo,Mexico,19.9786,-88.6850
13,926702,1995-01-01,Police (Mexico),"Government,Police",Mexico,Impose administrative sanctions,172,-5.0,Mexico,NaN,Mexico,28238094,6,The Associated Press Political Service,Tabasco,Jose Maria Morelos,Estado de Quintana Roo,Mexico,19.9786,-88.6850
14,926703,1995-01-01,Andrés Manuel López Obrador,"Protestors / Popular Opposition / Mobs,Dissident",Mexico,Refuse to yield,124,-4.0,Roberto Madrazo,"Center Left,Ideological,Government,(National) ...",Mexico,28238679,6,The Associated Press Political Service,Villahermosa,NaN,Distrito Federal,Mexico,19.4833,-99.1000
21,926711,1995-01-01,Main Opposition (Mexico),"Protestors / Popular Opposition / Mobs,Dissident",Mexico,Consider policy option,14,0.0,Carlos Salinas De Gortari,"Center Left,Ideological,(National) Major Party...",Mexico,28242696,1,The Washington Post,NaN,NaN,NaN,Mexico,19.4285,-99.1277
22,926715,1995-01-01,Main Opposition (Mexico),"Protestors / Popular Opposition / Mobs,Dissident",Mexico,Threaten,130,-4.4,Carlos Salinas De Gortari,"Center Left,Ideological,(National) Major Party...",Mexico,28242696,1,The Washington Post,NaN,NaN,NaN,Mexico,19.4285,-99.1277
23,926712,1995-01-01,Main Opposition (Mexico),"Protestors / Popular Opposition / Mobs,Dissident",Mexico,Consider policy option,14,0.0,Citizen (Mexico),"Social,General Population / Civilian / Social",Mexico,28242696,1,The Washington Post,Salinas,NaN,Estado de Tamaulipas,Mexico,26.2833,-98.8167
24,926716,1995-01-01,Main Opposition (Mexico),"Protestors / Popular Opposition / Mobs,Dissident",Mexico,Threaten,130,-4.4,Citizen (Mexico),"Social,General Population / Civilian / Social",Mexico,28242696,1,The Washington Post,Salinas,NaN,Estado de Tamaulipas,Mexico,26.2833,-98.8167
25,926713,1995-01-01,National Action Party (Mexico),Parties,Mexico,Express intent to meet or negotiate,36,4.0,Ernesto Zedillo,"Elite,Center Left,Ideological,Government,(Nati...",Mexico,28242696,3,The Washington Post,NaN,NaN,NaN,Mexico,19.4285,-99.1277
26,926714,1995-01-01,Party of the Democratic Revolution,"Ideological,(National) Major Party,Parties,Far...",Mexico,Accuse,112,-2.0,Carlos Salinas De Gortari,"Center Left,Ideological,(National) Major Party...",Mexico,28242696,4,The Washington Post,NaN,NaN,NaN,Mexico,19.4285,-99.1277
31,926734,1995-01-02,Protester (Mexico),"Protestors / Popular Opposition / Mobs,Dissident",Mexico,Demonstrate for leadership change,1411,-6.5,Governor (Mexico),"Government,Provincial,Local",Mexico,28236991,1,The Associated Press Political Service,Tabasco,Jose Maria Morelos,Estado de Quintana Roo,Mexico,19.9786,-88.6850


In [11]:
#completely removes numbers with apostrophes
#mex_clean['cameo_code'] = mex_clean['cameo_code'].str.replace(r"\'",'')

In [12]:
# #CAMEO codes to show which events are present in the dataframe
# explore = mex_clean['cameo_code'].unique()
# explore

In [13]:
mex_clean["cameo_code"].value_counts()

10      35242
40      17978
173     15101
20       9101
43       7984
        ...  
1324        1
0331        1
186         1
82          1
023         1
Name: cameo_code, Length: 338, dtype: int64

In [28]:
mex_test = mex_clean.loc[mex_clean["cameo_code"].isin(['2017-01-01'])]
mex_test.head()

,event_id,event_date,source_name,source_sectors,source_country,event_text,cameo_code,intensity,target_name,target_sectors,target_country,story_id,sentence_number,publisher,city,district,provice,country,lat,lon


In [37]:
mex_violence = mex_clean.loc[mex_clean["cameo_code"].isin([
180,181,182,183,184,185,186,190,191,192,193,194,195,202,1822,128,1823])]

In [38]:
mex_violence.shape

(17896, 20)

In [31]:
mex_violence["event_date"].unique

<bound method Series.unique of 325        1995-01-06
326        1995-01-06
336        1995-01-06
1351       1995-01-19
1436       1995-01-20
              ...    
16630623   2020-04-01
16634082   2020-04-04
16659004   2020-04-29
16659005   2020-04-29
16659006   2020-04-29
Name: event_date, Length: 17896, dtype: datetime64[ns]>

In [42]:
#standardize cartel events
cartels = {
    'Armed Gang (Sinaloa Cartel)': 'Sinaloa Cartel',
    'Armed Gang (Juárez Cartel)':'Juárez Cartel',
    'Organized Crime (Gulf Cartel)':'Gulf Cartel',
    'Armed Gang (Gulf Cartel)':'Gulf Cartel',
    'Drug Gang (Los Zetas Cartel)':'Los Zetas Cartel',
    'Hitman (Gulf Cartel)':'Gulf Cartel',
    'Hitman (Los Zetas Cartel)':'Los Zetas Cartel',
    'Criminal (Los Zetas Cartel)':'Los Zetas Cartel',
    'Drug Gang (Gulf Cartel)':'Gulf Cartel',
    'Armed Services Deserter (Los Zetas Cartel)':'Los Zetas Cartel',
    'Armed Gang (Knights Templar Cartel)':'Knights Templar Cartel'
}
mex_clean=mex_clean.replace(cartels)

In [44]:
#extract year to separate column
mex_clean['event_year'] = mex_clean['event_date'].dt.year

In [45]:
mex_clean.columns

Index(['event_id', 'event_date', 'source_name', 'source_sectors',
       'source_country', 'event_text', 'cameo_code', 'intensity',
       'target_name', 'target_sectors', 'target_country', 'story_id',
       'sentence_number', 'publisher', 'city', 'district', 'provice',
       'country', 'lat', 'lon', 'event_year'],
      dtype='object')

In [46]:
mex_clean.to_csv('../data/icews_mexico_violence.csv')